# Test 1

### Packages

In [1]:
import pandas as pd
from datetime import timedelta  
import os

os.chdir('/Users/tristanmayer/Documents/GitHub/taxi-time')
from src.preprocessing import *
from src.master import *

### Data

In [2]:
df1 = cleaning_airport_df('/Users/tristanmayer/Desktop/Supercase Eleven/data/0. Airport data/Airport_Data.csv')

/Users/tristanmayer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [3]:
df2 = get_df_of_obs1(df1)

In [4]:
df3 = get_target_values(df2)

In [5]:
## Creation des timestamps
## Cleaning of rows that font chier la bite
df1 = df1.loc[df1['aldt']!='aldt']
for col in ['aldt', 'aibt','aobt', 'atot']:
    df1[col] = pd.to_datetime(df1[col])

In [49]:
# number of planes in the airport at time_t (a date in string format)
# number of planes in block at time_t (a date in string format)
#### need to reconcile flights with plane (we only know details about flight so not the same line for arrival and departure from the stand -> see the dates regarding one stand)

# number of planes in movement at time_t (a date in string format)
## input: date as a string
## output: integer
def get_nb_of_planes_in_movement(time_t):
    t = pd.to_datetime(time_t)
    res = len(df1[((df1['aldt']<t) & (df1['aibt']>t))|
                  ((df1['aobt']<t) & (df1['atot']>t))])
    return res

# number of planes that have landed on the runway_R in the last M_min at time_t
## input: runway as a string; date as a string; timedelta as an integer (nb of minutes)
## output: integer
def get_nb_of_planes_runway_in_last_M_min(runway_R, time_t, M_min):
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    res = len(df1[(df1['runway']==runway_R) &
                  (df1['aldt']<t1) &
                  (df1['aldt']>t2)])
    return res


# number of planes that have stayed at the stand_S in the last M min at time_t
def get_nb_of_planes_stand_in_last_M_min(stand_S, time_t, M_min):
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    res = len(df1[(df1['stand']==stand_S) &
                  (df1['aibt']<t1) &
                  (df1['aibt']>t2)])
    return res

# average taxi-time of the planes that have landed on runway_R in the last M min at time_t
def average_taxitime_runway_last_X_min(runway_R, time_t, M_min): 
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    list_of_index = df1[(df1[df1['runway']==runway_R]) & 
                        (df1['aibt']<t1) & 
                        (df1['aibt']>t2)].index
    res = df3.loc[list_of_index][0].mean()
    return res

# average taxi-time of the planes that stayed at stand_S in the last M min at time_t
def average_taxitime_stand_last_X_min(stand_S, time_t, M_min): 
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    list_of_index = df1[(df1[df1['stand']==stand_S]) & 
                        (df1['aibt']<t1) & 
                        (df1['aibt']>t2)].index
    res = df3.loc[list_of_index][0].mean()
    return res

# taxi_time of the last plane that have landed on runway_R at time_t
def get_last_taxitime_runway(runway_R, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.iloc[df1[(df1['runway']==runway_R) & (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

# taxi_time of the last plane that have stayed at stand_S at time_t
def get_last_taxitime_stand(stand_S, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.iloc[df1[(df1['stand']==stand_S) & (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

    
# taxi_time of the last plane that have gone from runway_R to stand_S at time_t
def get_last_taxitime_path(runway_R, stand_S, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.iloc[df1[(df1['runway']==runway_R) &
                       (df1['stand']==stand_S) & 
                       (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

In [7]:
time_t = '2018-08-01 19:59:00'

In [5]:
df1.head()

,carrier,flight,acType,sto,runway,stand,aldt,eibt,cibt,aibt,chocks_on,eobt,aobt,atot
0,CAR1,NaN,MD88,undefined,NaN,STAND01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CAR1,NaN,MD88,undefined,NaN,STAND01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,CRJ/9,7/30/2018 4:18,NaN,STAND02,NaN,7/30/2018 4:18,NaN,NaN,NaN,NaN,NaN,NaN
3,CAR1,27,A350/9,7/30/2018 4:19,RUNWAY01,STAND03,NaN,NaN,NaN,NaN,NaN,7/30/2018 4:19,7/30/2018 4:18,7/30/2018 4:33
4,CAR1,2519,B737/9-WL,7/30/2018 4:45,RUNWAY02,STAND04,7/30/2018 4:25,7/30/2018 4:33,7/30/2018 4:33,7/30/2018 4:29,7/30/2018 4:30,NaN,NaN,NaN


In [30]:
t = pd.to_datetime(time_t)
len(df1[((df1['aldt']<t) & (df1['aibt']>t))|
        ((df1['aobt']<t) & (df1['atot']>t))])

36

In [31]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 300)
len(df1[(df1['runway']=='RUNWAY01')&
        (df1['aldt']<t1) &
        (df1['aldt']>t2)])

0

In [19]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 300)
res = len(df1[(df1['stand']=='STAND03') &
                  (df1['aibt']<t1) &
                  (df1['aibt']>t2)])
res

1

In [24]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 30000)
list_of_index = df1[(df1['stand']=='STAND03') & 
                    (df1['aibt']<t1) & 
                    (df1['aibt']>t2)].index
res = df3.loc[list_of_index][0].mean()
res

576.0

In [46]:
t1 = pd.to_datetime(time_t)
df3.iloc[df1[(df1['runway']=='RUNWAY01') & (df1['aibt']<t1)]['aibt'].idxmax()][0]
         
         
#.sort_values('aibt', ascending=False).iloc[0].name][0]

300.0

In [28]:
df1[(df1['stand']=='STAND03')]

,carrier,flight,acType,sto,runway,stand,aldt,eibt,cibt,aibt,chocks_on,eobt,aobt,atot
3,CAR1,27,A350/9,7/30/2018 4:19,RUNWAY01,STAND03,NaT,NaN,NaN,NaT,NaN,7/30/2018 4:19,2018-07-30 04:18:00,2018-07-30 04:33:00
17,CAR1,1943,B777/2-LR,7/30/2018 8:14,RUNWAY01,STAND03,2018-07-30 09:21:00,7/30/2018 9:28,7/30/2018 9:27,2018-07-30 09:28:00,NaN,NaN,NaT,NaT
1175,CAR1,185,B777/2-LR,7/30/2018 19:20,RUNWAY04,STAND03,NaT,NaN,NaN,NaT,NaN,7/30/2018 19:20,2018-07-30 19:19:00,2018-07-30 19:40:00
1431,CAR1,65,A330/3,7/30/2018 21:33,RUNWAY06,STAND03,2018-07-30 21:10:00,7/30/2018 21:18,7/30/2018 21:19,2018-07-30 21:18:00,NaN,NaN,NaT,NaT
2189,CAR1,101,A330/3,7/31/2018 2:23,RUNWAY04,STAND03,NaT,NaN,NaN,NaT,NaN,7/31/2018 3:20,2018-07-31 03:17:00,2018-07-31 03:35:00
2936,CAR1,295,B777/2-LR,7/31/2018 15:49,RUNWAY01,STAND03,NaT,NaN,NaN,NaT,NaN,7/31/2018 15:49,2018-07-31 15:44:00,2018-07-31 15:58:00
3042,CAR1,83,A330/3,7/31/2018 16:52,RUNWAY10,STAND03,2018-07-31 16:39:00,7/31/2018 16:49,7/31/2018 16:46,2018-07-31 16:47:00,NaN,NaN,NaT,NaT
3587,CAR1,33,A330/2,7/31/2018 22:30,RUNWAY08,STAND03,2018-07-31 22:12:00,7/31/2018 22:24,7/31/2018 22:26,2018-07-31 22:31:00,NaN,NaN,NaT,NaT
4326,CAR1,54,A330/2,8/1/2018 2:13,RUNWAY01,STAND03,NaT,NaN,NaN,NaT,NaN,8/1/2018 3:50,2018-08-01 03:42:00,2018-08-01 03:55:00
5261,CAR1,295,B777/2-LR,8/1/2018 15:49,RUNWAY04,STAND03,NaT,NaN,NaN,NaT,NaN,8/1/2018 15:49,2018-08-01 15:46:00,2018-08-01 16:00:00
